In [1]:
import math
import os

import polars as pl
import datetime as dt

from estimator.estimator import *
from estimator.estimator.nd import NoiseDistribution

In [2]:
jobs = 16
expect_security = 128

# bits - log poly length
Q27_10 = 134215681
Q27_11 = 134176769
Q27_20 = 132120577

Q29_10 = 536856577
Q29_11 = 536813569

Q51_11 = 2251799813640193
Q54_11 = 18014398509404161
Q56_11 = 72057594037641217

In [3]:
data_path = "./security_result.parquet"
# data_path = "./test.parquet"

schema = {
    "dimension": pl.UInt64,
    "modulus": pl.UInt64,
    "secret_distr": pl.String,
    "secret_stddev": pl.Float64,
    "noise_stddev": pl.Float64,
    "security_bits": pl.Float64,
    "datetime": pl.Datetime,
}

if not os.path.exists(data_path):
    df = pl.DataFrame(None, schema)
    df.write_parquet(data_path)

df = pl.read_parquet(data_path)
print(
    df.filter(pl.col("security_bits") >= expect_security)
    .sort(["dimension", "modulus"])
    .head()
)

shape: (5, 7)
┌───────────┬──────────────┬──────────────┬──────────────┬─────────────┬─────────────┬─────────────┐
│ dimension ┆ modulus      ┆ secret_distr ┆ secret_stdde ┆ noise_stdde ┆ security_bi ┆ datetime    │
│ ---       ┆ ---          ┆ ---          ┆ v            ┆ v           ┆ ts          ┆ ---         │
│ u64       ┆ u64          ┆ str          ┆ ---          ┆ ---         ┆ ---         ┆ datetime[μs │
│           ┆              ┆              ┆ f64          ┆ f64         ┆ f64         ┆ ]           │
╞═══════════╪══════════════╪══════════════╪══════════════╪═════════════╪═════════════╪═════════════╡
│ 512       ┆ 2048         ┆ Binary       ┆ 0.5          ┆ 0.83        ┆ 128.025423  ┆ 2024-11-27  │
│           ┆              ┆              ┆              ┆             ┆             ┆ 16:46:26.99 │
│           ┆              ┆              ┆              ┆             ┆             ┆ 4831        │
│ 512       ┆ 2048         ┆ Ternary      ┆ 0.707107     ┆ 0.63        ┆ 128.

In [4]:
def write_to_data(
    dimension: int,
    modulus: int,
    secret_type: str,
    secret_distr: NoiseDistribution,
    noise_stddev,
    security,
):
    new = pl.DataFrame(
        {
            "dimension": [dimension],
            "modulus": [modulus],
            "secret_distr": [secret_type],
            "secret_stddev": [secret_distr.stddev],
            "noise_stddev": [noise_stddev],
            "security_bits": [security],
            "datetime": [dt.datetime.now()],
        },
        schema,
    )
    print(new)
    df.extend(new)
    df.write_parquet(data_path)

In [5]:
def print_uncheck(
    dimension: int,
    modulus: int,
    secret_type: str,
    secret_distr: NoiseDistribution,
    noise_stddev,
    security,
    pat: str,
):
    uncheck = pl.DataFrame(
        {
            "dimension": [dimension],
            "modulus": [modulus],
            "secret_distr": [secret_type],
            "secret_stddev": [secret_distr.stddev],
            "noise_stddev": [noise_stddev],
            "security_bits": [pat.format(security)],
        },
    )
    print(uncheck)

In [6]:
def check_security(
    dimension: int,
    modulus: int,
    secret_distr: NoiseDistribution,
    noise_stddev,
    is_quantum=False,
):
    param = LWE.Parameters(
        n=dimension, q=modulus, Xs=secret_distr, Xe=ND.DiscreteGaussian(noise_stddev)
    )
    print(param)

    try:
        result = LWE.estimate(
            param,
            red_cost_model=(RC.LaaMosPol14 if is_quantum else RC.BDGL16),
            deny_list=(
                "arora-gb",
                "bkw",
                "bdd_hybrid",
                "bdd_mitm_hybrid",
                "dual_hybrid",
                "dual_mitm_hybrid",
            ),
            jobs=jobs,
        )
    except Exception as err:
        print("err=", err)
        print("Error Occur!")
    else:
        return min([math.log(res.get("rop", 0), 2) for res in result.values()])

In [7]:
def check_security_with_data(
    dimension: int,
    modulus: int,
    secret_type: str,
    secret_distr: NoiseDistribution,
    noise_stddev,
    force_run_estimate: bool = False,
):
    if force_run_estimate:
        security = check_security(dimension, modulus, secret_distr, noise_stddev)
        write_to_data(
            dimension, modulus, secret_type, secret_distr, noise_stddev, security
        )
        return

    temp = df.filter(dimension=dimension, modulus=modulus, secret_distr=secret_type)
    exact = temp.filter(noise_stddev=noise_stddev)

    if exact.is_empty():
        high = temp.filter(pl.col("security_bits") >= expect_security).sort(
            "security_bits"
        )

        if (not high.is_empty()) and high.item(0, "noise_stddev") <= noise_stddev:
            print_uncheck(
                dimension,
                modulus,
                secret_type,
                secret_distr,
                noise_stddev,
                high.item(0, "security_bits"),
                ">{}",
            )
            return

        low = temp.filter(pl.col("security_bits") < expect_security).sort(
            "security_bits", descending=True
        )

        if (not low.is_empty()) and low.item(0, "noise_stddev") >= noise_stddev:
            print_uncheck(
                dimension,
                modulus,
                secret_type,
                secret_distr,
                noise_stddev,
                low.item(0, "security_bits"),
                "<{}",
            )
            return

        security = check_security(dimension, modulus, secret_distr, noise_stddev)

        write_to_data(
            dimension, modulus, secret_type, secret_distr, noise_stddev, security
        )
    else:
        print(exact)

In [24]:
# LWE
n1 = 512
q1 = 2048


lwe1_key_type = "Binary"
lwe1_noise_stddev = 0.83
lwe1_key_distribution = ND.SparseBinary(n1 // 2, n1)

# lwe1_key_type = "Ternary"
# lwe1_noise_stddev = 0.63
# lwe1_key_distribution = ND.SparseTernary(n1 // 4, n1 // 4, n1)

In [25]:
check_security_with_data(
    n1, q1, lwe1_key_type, lwe1_key_distribution, lwe1_noise_stddev
)

shape: (1, 7)
┌───────────┬─────────┬──────────────┬───────────────┬──────────────┬───────────────┬──────────────┐
│ dimension ┆ modulus ┆ secret_distr ┆ secret_stddev ┆ noise_stddev ┆ security_bits ┆ datetime     │
│ ---       ┆ ---     ┆ ---          ┆ ---           ┆ ---          ┆ ---           ┆ ---          │
│ u64       ┆ u64     ┆ str          ┆ f64           ┆ f64          ┆ f64           ┆ datetime[μs] │
╞═══════════╪═════════╪══════════════╪═══════════════╪══════════════╪═══════════════╪══════════════╡
│ 512       ┆ 2048    ┆ Binary       ┆ 0.5           ┆ 0.83         ┆ 128.025423    ┆ 2024-11-27   │
│           ┆         ┆              ┆               ┆              ┆               ┆ 16:46:26.994 │
│           ┆         ┆              ┆               ┆              ┆               ┆ 831          │
└───────────┴─────────┴──────────────┴───────────────┴──────────────┴───────────────┴──────────────┘


In [26]:
# BSK 1
N1 = 1024
Q1 = Q27_10


# bsk1_key_type = "Binary"
# bsk1_stddev = 4.49
# bsk1_key_distribution = ND.SparseBinary(N1 // 2, N1)

bsk1_key_type = "Ternary"
bsk1_stddev = 3.2
bsk1_key_distribution = ND.SparseTernary(N1 // 4, N1 // 4, N1)

In [27]:
check_security_with_data(N1, Q1, bsk1_key_type, bsk1_key_distribution, bsk1_stddev)

shape: (1, 7)
┌───────────┬───────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┐
│ dimension ┆ modulus   ┆ secret_distr ┆ secret_stdde ┆ noise_stddev ┆ security_bit ┆ datetime     │
│ ---       ┆ ---       ┆ ---          ┆ v            ┆ ---          ┆ s            ┆ ---          │
│ u64       ┆ u64       ┆ str          ┆ ---          ┆ f64          ┆ ---          ┆ datetime[μs] │
│           ┆           ┆              ┆ f64          ┆              ┆ f64          ┆              │
╞═══════════╪═══════════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════════╡
│ 1024      ┆ 134215681 ┆ Ternary      ┆ 0.707107     ┆ 3.2          ┆ 128.034311   ┆ 2024-11-27   │
│           ┆           ┆              ┆              ┆              ┆              ┆ 17:00:10.450 │
│           ┆           ┆              ┆              ┆              ┆              ┆ 119          │
└───────────┴───────────┴──────────────┴──────────────┴──────────────┴───────

In [40]:
n2 = 1024


ksk_type = "Binary"
ksk_noise_stddev = 4.49
ksk_distribution = ND.SparseBinary(n2 // 2, n2)

# ksk_type = "Ternary"
# ksk_noise_stddev = 3.2
# ksk_distribution = ND.SparseTernary(n2 // 4, n2 // 4, n2)

In [41]:
check_security_with_data(n2, Q1, ksk_type, ksk_distribution, ksk_noise_stddev)

shape: (1, 7)
┌───────────┬───────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┐
│ dimension ┆ modulus   ┆ secret_distr ┆ secret_stdde ┆ noise_stddev ┆ security_bit ┆ datetime     │
│ ---       ┆ ---       ┆ ---          ┆ v            ┆ ---          ┆ s            ┆ ---          │
│ u64       ┆ u64       ┆ str          ┆ ---          ┆ f64          ┆ ---          ┆ datetime[μs] │
│           ┆           ┆              ┆ f64          ┆              ┆ f64          ┆              │
╞═══════════╪═══════════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════════╡
│ 1024      ┆ 134215681 ┆ Binary       ┆ 0.5          ┆ 4.49         ┆ 128.247523   ┆ 2024-11-27   │
│           ┆           ┆              ┆              ┆              ┆              ┆ 16:46:29.486 │
│           ┆           ┆              ┆              ┆              ┆              ┆ 929          │
└───────────┴───────────┴──────────────┴──────────────┴──────────────┴───────

In [42]:
N2 = 2048
Q2 = Q54_11


# bsk2_key_type = "Binary"
# bsk2_nosie_stddev = 4.63
# bsk2_key_distribution = ND.SparseBinary(N2 // 2, N2)

bsk2_key_type = "Ternary"
bsk2_nosie_stddev = 3.4
bsk2_key_distribution = ND.SparseTernary(N2 // 4, N2 // 4, N2)

In [43]:
check_security_with_data(
    N2, Q2, bsk2_key_type, bsk2_key_distribution, bsk2_nosie_stddev
)

shape: (1, 7)
┌───────────┬──────────────┬──────────────┬──────────────┬─────────────┬─────────────┬─────────────┐
│ dimension ┆ modulus      ┆ secret_distr ┆ secret_stdde ┆ noise_stdde ┆ security_bi ┆ datetime    │
│ ---       ┆ ---          ┆ ---          ┆ v            ┆ v           ┆ ts          ┆ ---         │
│ u64       ┆ u64          ┆ str          ┆ ---          ┆ ---         ┆ ---         ┆ datetime[μs │
│           ┆              ┆              ┆ f64          ┆ f64         ┆ f64         ┆ ]           │
╞═══════════╪══════════════╪══════════════╪══════════════╪═════════════╪═════════════╪═════════════╡
│ 2048      ┆ 180143985094 ┆ Ternary      ┆ 0.707107     ┆ 3.4         ┆ 128.101037  ┆ 2024-11-27  │
│           ┆ 04161        ┆              ┆              ┆             ┆             ┆ 17:00:14.04 │
│           ┆              ┆              ┆              ┆             ┆             ┆ 8246        │
└───────────┴──────────────┴──────────────┴──────────────┴─────────────┴─────